In [48]:
import pandas as pd
import numpy as np
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
# import datetime as dt
from datetime import datetime, timedelta
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [49]:
raw_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [50]:
main_data = raw_data[0]
main_data['Symbol'] = main_data['Symbol'].str.replace('.', '-')
symbols_list = main_data['Symbol'].unique().tolist()
today = datetime.now().date()
end_date = today - timedelta(days=2)
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)

In [51]:
yf_df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack()
yf_df.index.names = ['date', 'ticker']
yf_df.columns = yf_df.columns.str.lower()

[*********************100%%**********************]  503 of 503 completed


In [52]:
yf_df['garman_klass_vol'] = ((np.log(yf_df['high']) - np.log(yf_df['low']))**2)/2-((2*np.log(2)-1)*(np.log(yf_df['adj close'])-np.log(yf_df['open']))**2)
yf_df['rsi'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))
yf_df['bb_low'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
yf_df['bb_mid'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
yf_df['bb_high'] = yf_df.groupby('ticker')['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])


## Definiting Fynction to obtain the ATR

In [53]:
def get_atr(df):
    atr = pandas_ta.atr(high=df['high'],
                        low=df['low'],
                        close=df['close'],
                        length=14 # standard lenght for calculating ATR but it can be changed based on the requirements
                        )
    atr_mean = atr.mean()
    atr_std = atr.std()
    normalized_atr = (atr - atr_mean) / atr_std
    return normalized_atr

In [54]:
yf_df['atr'] = yf_df.groupby('ticker', group_keys=False).apply(get_atr)

## Defining Function to calculate MACD

In [55]:
def get_macd(i):
    try:
        macd = pandas_ta.macd(close=i, length= 20).iloc[:,0]
        macd_mean = macd.mean()
        macd_std = macd.std()
        normalized_macd = (macd - macd_mean) / macd_std
        return normalized_macd
    except Exception as e:
        print(f'error : {e}')
        print(f'what is causing the error {i}')

In [63]:
type(yf_df['adj close'])

pandas.core.series.Series

In [57]:
yf_df['macd'] = yf_df.groupby('ticker', group_keys=False)['adj close'].apply(get_macd)

error : 'NoneType' object has no attribute 'iloc'
what is causing the error date        ticker
2023-10-04  VLTO      77.800003
2023-10-05  VLTO      74.449997
2023-10-06  VLTO      77.980003
2023-10-09  VLTO      74.610001
2023-10-10  VLTO      75.000000
2023-10-11  VLTO      74.470001
2023-10-12  VLTO      74.230003
2023-10-13  VLTO      71.849998
2023-10-16  VLTO      74.209999
2023-10-17  VLTO      74.480003
2023-10-18  VLTO      72.449997
2023-10-19  VLTO      70.290001
2023-10-20  VLTO      69.930000
2023-10-23  VLTO      71.900002
2023-10-24  VLTO      72.010002
2023-10-25  VLTO      70.790001
2023-10-26  VLTO      70.000000
2023-10-27  VLTO      67.989998
2023-10-30  VLTO      68.360001
2023-10-31  VLTO      69.000000
2023-11-01  VLTO      67.510002
Name: VLTO, dtype: float64


In [58]:
yf_df

adj close       close        high         low        open  \
date       ticker                                                               
2013-11-04 A        33.627903   36.630901   36.809727   36.444920   36.759659   
           AAL      21.409649   22.709999   23.900000   22.570000   22.600000   
           AAPL     16.418627   18.812500   18.815001   18.528929   18.610714   
           ABBV     32.370590   48.759998   49.189999   48.000000   49.189999   
           ABT      30.490248   36.980000   37.020000   36.689999   36.980000   
...                       ...         ...         ...         ...         ...   
2023-11-01 YUM     121.300003  121.300003  122.480003  119.529999  121.879997   
           ZBH     105.070000  105.070000  105.209999  103.510002  104.519997   
           ZBRA    197.990005  197.990005  212.500000  194.589996  209.570007   
           ZION     31.020000   31.020000   31.129999   30.330000   30.740000   
           ZTS     151.440002  151.440002  157.000000  151.029999  156.419998   

                        volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2013-11-04 A         1971040.0         -0.003013        NaN       NaN   
           AAL      19962000.0          0.000508        NaN       NaN   
           AAPL    244627600.0         -0.005950        NaN       NaN   
           ABBV      4095600.0         -0.067337        NaN       NaN   
           ABT       5645300.0         -0.014345        NaN       NaN   
...                        ...               ...        ...       ...   
2023-11-01 YUM       2788100.0          0.000288  46.889197  4.769195   
           ZBH       1503100.0          0.000122  38.026745  4.620332   
           ZBRA       822900.0          0.002628  32.598621  5.283557   
           ZION      2261800.0          0.000307  43.413219  3.365911   
           ZTS       7269900.0          0.000347  24.661370  5.032226   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2013-11-04 A            NaN       NaN       NaN       NaN  
           AAL          NaN       NaN       NaN       NaN  
           AAPL         NaN       NaN       NaN       NaN  
           ABBV         NaN       NaN       NaN       NaN  
           ABT          NaN       NaN       NaN       NaN  
...                     ...       ...       ...       ...  
2023-11-01 YUM     4.791868  4.814540  0.558270 -0.962262  
           ZBH     4.670407  4.720483 -0.246480 -1.283439  
           ZBRA    5.354364  5.425171  0.383412 -1.337473  
           ZION    3.513926  3.661940  0.304610 -1.247455  
           ZTS     5.125014  5.217803  0.910193 -3.193959  

[1234434 rows x 13 columns]

In [62]:
for i in yf_df.index.get_level_values('ticker').unique():
    print(i)


A
AAL
AAPL
ABBV
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BG
BIIB
BIO
BK
BKNG
BKR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CF
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTRA
CTSH
CVS
CVX
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOV
DPZ
DRI
DTE
DUK
DVA
DVN
DXCM
EA
EBAY
ECL
ED
EFX
EG
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
EQT
ES
ESS
ETN
ETR
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FLT
FMC
FRT
FSLR
FTNT
GD
GE
GEN
GILD
GIS
GL
GLW
GM
GNRC
GOOG
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HOLX
HON
HPQ
HRL
HSIC
HST
HSY
HUBB
HUM
IBM
ICE
IDXX
IEX
IFF
ILMN
INCY
INTC
INTU
IP
IPG
IQV
IRM
ISRG
IT
ITW
IVZ
J
JBHT
JCI
JKHY
JNJ
JNPR
JPM
K
KDP
KEY
K